In [1]:
import torch
from PIL import Image
from transformers import AutoImageProcessor, AutoModel, AutoTokenizer, AutoProcessor, AutoModelForSeq2SeqLM
import numpy as np
import json
from datasets import load_dataset
import os
from tqdm import tqdm

/opt/conda/envs/mmqa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-19 11:23:17.291162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-19 11:23:17.580339: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-19 11:23:17.582323: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-19 11:23:18.082968: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is op

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained("openai/clip-vit-base-patch16").to(device)
processor = AutoImageProcessor.from_pretrained("openai/clip-vit-base-patch16")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch16")
print(device)

cuda


/opt/conda/envs/mmqa/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
import os
print(os.getcwd())

/home/filiprejmus/multimodalqa/baselines/llm_qa


In [4]:
import json
from datasets import Dataset, Features, Image, Value
from PIL import Image as PILImage
import os

def load_and_validate_image(image_path):
    try:
        with PILImage.open(image_path) as img:
            img.verify()  # Verify that it's a valid image
        return image_path
    except (IOError, SyntaxError):
        return None  # Return None for corrupt or missing images

def load_dataset(metadata_file, image_folder):
    data = []
    
    with open(metadata_file, 'r') as f:
        for line in f:
            item = json.loads(line)
            image_path = os.path.join(image_folder, item['path'])
            
            # Validate and load the image
            validated_image_path = load_and_validate_image(image_path)
            
            if validated_image_path:
                item['image'] = validated_image_path
                data.append(item)
            else:
                print(f"Warning: Image not found or corrupt: {image_path}")
    
    return data

# Set your file paths
metadata_file = '../downloads/MMQA/MMQA_images.jsonl'
image_folder = '../downloads/MMQA/final_dataset_images'

# Load and process the data
dataset_dict = load_dataset(metadata_file, image_folder)

# Define the features of your dataset
features = Features({
    'title': Value('string'),
    'url': Value('string'),
    'id': Value('string'),
    'path': Value('string'),
    'image': Image()
})

# Create the dataset
dataset = Dataset.from_dict({k: [d[k] for d in dataset_dict] for k in dataset_dict[0].keys()}, features=features)

# Print some information about the dataset
print(dataset)
print(f"Number of samples: {len(dataset)}")

# You can now use this dataset or save it
# dataset.save_to_disk("my_image_dataset")
# Or push to the Hugging Face Hub
# dataset.push_to_hub("username/dataset-name")

/opt/conda/envs/mmqa/lib/python3.11/site-packages/PIL/Image.py:3368: DecompressionBombWarning: Image size (129967200 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Dataset({
    features: ['title', 'url', 'id', 'path', 'image'],
    num_rows: 57052
})
Number of samples: 57052


In [5]:
# Load model directly
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

In [11]:
inputs = processor(dataset[0:10]['image'], return_tensors="pt").to("cuda")

In [12]:
out = model.generate(**inputs)

/opt/conda/envs/mmqa/lib/python3.11/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [16]:
print(processor.decode(out[2], skip_special_tokens=True))

smiling woman with blonde hair and blue sweater standing in front of water


In [33]:
import json
from tqdm import tqdm
import torch
from transformers import AutoProcessor, AutoModelForCausalLM

# Load your model and processor
processor = AutoProcessor.from_pretrained("your_model_name")
model = AutoModelForCausalLM.from_pretrained("your_model_name").to("cuda")

def generate_descriptions(dataset, start_index, batch_size=100):
    end_index = min(start_index + batch_size, len(dataset))
    batch = dataset[start_index:end_index]
    descriptions = []
    try:
        inputs = processor(images=batch['image'], return_tensors="pt", padding=True).to("cuda")
        with torch.no_grad():
            out = model.generate(**inputs)
        
        for o in out:
            description = processor.decode(o, skip_special_tokens=True)
            descriptions.append(description)
        return descriptions
    except Exception as e:
        print(f"Error processing batch starting at index {start_index}: {str(e)}")
        return [None] * (end_index - start_index)

def process_dataset(dataset, output_file, start_index=0, batch_size=100):
    with open(output_file, 'a') as f:
        for i in tqdm(range(start_index, len(dataset), batch_size), total=(len(dataset)-start_index)//batch_size + 1):
            end_index = min(i + batch_size, len(dataset))
            descriptions = generate_descriptions(dataset, i, batch_size)
            
            for j, description in enumerate(descriptions):
                item = dataset[i + j]
                if description is not None:
                    result = {
                        "id": item['id'],
                        "title": item['title'],
                        "url": item['url'],
                        "image_path": item['image'],
                        "description": description
                    }
                    json.dump(result, f)
                    f.write('\n')
                    f.flush()  # Ensure the write is committed to disk

            print(f"Processed items {i} to {end_index-1}. Last processed ID: {dataset[end_index-1]['id']}")

# Set your output file
output_file = "descriptions_progress.jsonl"

# Process the dataset
process_dataset(dataset, output_file, start_index=0, batch_size=100)

print(f"Processing complete. Results saved to {output_file}")

/opt/conda/envs/mmqa/lib/python3.11/site-packages/PIL/Image.py:3368: DecompressionBombWarning: Image size (129967200 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [ ]:
dataset = dataset.cast_column("description", Value("string"))


In [ ]:
print(dataset)
print(f"Number of samples: {len(dataset)}")
print(f"Sample entry: {dataset[0]}")

In [29]:
print(dataset)

Dataset({
    features: ['title', 'url', 'id', 'path', 'image'],
    num_rows: 57052
})


In [30]:
print(dataset[0])

{'title': 'Republican Party (United States)', 'url': 'https://en.wikipedia.org/wiki/Republican_Party_(United_States)', 'id': '35b31d9b4f723f806fd32662ef29edf7', 'path': '35b31d9b4f723f806fd32662ef29edf7.jpg', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=227x227 at 0x7F21FEFB3E90>}
